# In this notebook, I will describe how to use variation of parameters to estimate a slater-type orbital as a linear combination of gaussian-type orbitals
## Specifically, I will consider the caes of a hydrogen atom

### Previous work:
Thankfully, the overlap $S$ matrix, as well as the kinetic energy $T$ matrix and the coulomb potential energy $V$ matrix has been calculated for me in terms of the standard deviations of these gaussians:
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\begin{align*}
S_{pq} &= \left ( \frac{\pi}{\alpha_p + \alpha_q} \right ) ^ {3/2}\\
T_{pq} &= 3 \frac{\alpha_p \alpha_q \pi^{3/2}}{\left(\alpha_p + \alpha_q\right)^{5/2}}\\
V_{pq} &= -\frac{2\pi}{\alpha_p + \alpha_q}
\end{align*}$$

### My work:
The slater-type orbital we would like to approximate is represented as a linear combination of the gaussian type orbitals:
$$
\begin{align*}
\ket{s} &= \displaystyle\sum_i C_i \ket{i}\\
\ket{i} &= \exp\left(\alpha_i r^2\right)
\end{align*}
$$
It remains to optimize the values of the coefficients $C_i$ and the parameters in the gaussians $\alpha_i$. I will do this with gradient descent.

### How it works:
The variational principle states that for any state with a variational parameter, nature will choose the value of that parameter that *minimizes* the energy. Thus, to optimize the linear combination of gaussians to most faithfully represent the slater-type orbital, the energy of this linear combination of gaussians must be minimized. This reduces the quantum mechanics problem to a more abstract optimization problem.

In [1]:
#First, I import the necessary libraries and set constants with regards to the learning parameters
import numpy as np
import math

PI_3_2 = math.pi ** 1.5
N = 6 #number of terms used in linear combination

#These can be tuned; I just found that these values work well.
ALPHA_LEARNING_RATE = 0.1
COEFF_LEARNING_RATE = 0.1
DELTA_A = 0.001
DELTA_C = 0.001

NUM_ITERS = 10 ** 4

Then, I set up the functions for calculating the energy as described above. It is important to note here that the energy of a linear combination of states is defined as:
$$
\begin{equation*}
E_\psi = \frac{\bra{\psi} \hat{H} \ket{\psi}}{\bra{\psi}\ket{\psi}}
\end{equation*}
$$

In [2]:
def calc_overlap(ap, aq):
    return PI_3_2 / ((ap + aq) ** 1.5)

def calc_KE(ap, aq):
    return 3 * PI_3_2 * (ap * aq) / (ap + aq) ** 2.5

def calc_PE(ap, aq):
    return -2 * math.pi / (ap + aq)

def calc_single_E(ap, aq):
    return calc_KE(ap, aq) + calc_PE(ap, aq)

def calc_norm(c, a):
    o = 0
    for i in range(N):
        for j in range(N):
            o += c[i] * c[j] * calc_overlap(a[i], a[j])
    return o

def calc_composite_E(c, a):
     #E = \sum_i \sum_j C_i C_j <i | H | j> / <i | j>
     E = 0
     o = 0
     for i in range(N):
         for j in range(N):
             E += c[i] * c[j] * calc_single_E(a[i], a[j])
     return E / calc_norm(c, a)

Now, I will implement the gradient descent algorithm. How it works is it calculates the direction of steepest descent for each of the parameters (each value of $\alpha$ and $C$) using a crude method of calculating the derivative (simply taking the slope of the local secant line as the slope of the tangent line). It then adjusts them in that direction an amount scaled by the learning rate parameters. Note: if alpha ever goes negative, the gaussian it represents now blows up to infinity, and it no longer represents a physical state. This would break all physical sense and all our calculations, so to prevent this from happening, I scale the change to alpha by the value of alpha itself. This way, it will never subtract something greater than alpha from alpha, keeping its value always positive.

In [3]:
def step(c, a):
    #Vary the parameters to see the direction of steepest descent.
    #Start with alphas and then move on to coefficients
    E_0 = calc_composite_E(c, a)
    #print(E_0)
    #Computing alphas
    gradients_a = []
    gradients_c = []
    for i in range(N):
        a[i] += DELTA_A
        gradients_a.append((calc_composite_E(c, a) - E_0) / DELTA_A)
        a[i] -= DELTA_A
    for i in range(N):
        c[i] += DELTA_C
        gradients_c.append((calc_composite_E(c, a) - E_0) / DELTA_C)
        c[i] -= DELTA_C
    a -= gradients_a * a # scaled by the alpha values so they never go negative
    c -= gradients_c
    return c, a

Now, I assign random values to the starting parameters, and let the code run!

In [4]:
c = np.random.rand(N)
a = np.random.rand(N)

for i in range(NUM_ITERS):
    c, a = step(c, a)
    if i % int(NUM_ITERS / 10) == 0:
        print(c, a, calc_composite_E(c, a))

[0.42644667 0.22109692 0.42759154 0.02041894] [0.99631022 0.38190557 0.17738582 0.95253861] -0.47656383273406483
[0.60723721 0.59799669 0.17585582 0.76451283] [4.81627174 0.25848006 0.0883374  0.92006116] -0.4977516011814803
[0.62730445 0.72635589 0.1630477  0.88693872] [6.05424727 0.26502342 0.08305094 1.04699258] -0.49851646686248074
[0.58393023 0.73583357 0.1693266  0.85887725] [6.80698246 0.28131638 0.08584556 1.14006673] -0.4987454947774738
[0.5398204  0.72114419 0.18023221 0.81036239] [7.36737242 0.29686864 0.08949457 1.21736254] -0.49882400506126645
[0.50855982 0.70951987 0.18939183 0.77464495] [7.82355225 0.3094898  0.09235219 1.2804832 ] -0.49886103969252066
[0.48769324 0.70302653 0.19592563 0.75177511] [8.20386995 0.31909655 0.09438427 1.33064889] -0.4988874810769719
[0.47304248 0.69931083 0.20071142 0.73641826] [8.52300555 0.32650889 0.09587606 1.37087347] -0.4989080298235916
[0.46217572 0.69698147 0.204391   0.72538661] [8.79235038 0.3323906  0.09702134 1.40368866] -0.49892

The values should converge to -0.5, as that is the theoretical limit, as determined without using approximations. Note: re-running the program will produce slightly different results, since the initial parameters are randomly set.